In [1]:
import pandas as pd

data = pd.read_parquet("../pipelines/data/db_pull_calibrated.parquet")
data

,timestamp,AccelerometerUncalibrated_x,AccelerometerUncalibrated_y,AccelerometerUncalibrated_z,GyroscopeUncalibrated_x,GyroscopeUncalibrated_y,GyroscopeUncalibrated_z,MagnetometerUncalibrated_x,MagnetometerUncalibrated_y,MagnetometerUncalibrated_z,activity,hash,person
0,2023-03-16 15:50:20.660,0.048859,-0.124008,-0.177597,0.227197,-1.061233,1.024258,-30.409225,203.701462,-119.364014,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
1,2023-03-16 15:50:20.670,0.121765,0.064194,-0.077682,0.286300,-1.063583,1.073323,-30.016541,203.783813,-119.510849,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
2,2023-03-16 15:50:20.680,0.109436,0.197830,0.036087,0.203359,-0.958838,0.973699,-29.712540,203.468811,-120.108040,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
3,2023-03-16 15:50:20.690,0.045609,0.283218,0.263519,-0.092112,-0.879174,0.795639,-29.202713,202.872742,-119.847961,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
4,2023-03-16 15:50:20.700,-0.050598,0.372131,0.444260,-0.567339,-0.878459,0.543531,-28.727325,203.102036,-119.527252,Rennen,040d27bca3b84ad198a2aa1129e4c8ac,Manjavy_Kirupa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4314652,2023-03-12 13:25:03.670,13.520895,-19.524834,3.057395,-1.111286,-2.552439,-3.282057,-60.840000,46.079998,63.119999,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314653,2023-03-12 13:25:03.680,8.350317,-17.924549,2.788287,-0.892718,-3.440760,-2.869601,-60.840000,44.820000,64.260002,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314654,2023-03-12 13:25:03.690,3.929424,-15.261121,2.180160,-0.712513,-3.864456,-2.479747,-61.739998,43.079998,65.639999,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess
4314655,2023-03-12 13:25:03.700,0.114922,-11.732846,2.489970,-0.468900,-3.782966,-2.111150,-61.739998,42.299999,67.379997,Rennen,ff34adc19eb8458fba1a668b8a7a9424,Tobias_Buess


In [2]:
data.value_counts(["hash", "activity", "person"]).head(10)

hash                              activity    person         
31ee3891f5ad49b1a989abec4dd30816  Sitzen      Ognjen_Colovic     121884
44c4e5f5f7df4203aca2f4e17afc6f35  Velofahren  Gabriel_Torres     105252
fc4955e4028d4e568ea29e35f8dfbeb9  Laufen      Ognjen_Colovic     100446
a212351e846d4da099db1a73032efe43  Stehen      Florin_Barbisch     95213
517458b7423a4f36920d4d95c53ad540  Sitzen      Yvo_Keller          95001
4eb98a019979402c87f4e0038a8ccf45  Laufen      Florin_Barbisch     81102
1c209f6ac3a546c498ca73ebb195611a  Sitzen      Florin_Barbisch     66210
e5236dad903942eabf03264e26c6d40f  Rennen      Florin_Barbisch     63825
a62bc673935e435ba7be437144e6827e  Sitzen      Tobias_Buess        63718
2c739c059b9f4632966a704e6970905e  Sitzen      Gabriel_Torres      61886
Name: count, dtype: int64

In [3]:
data = data[data["hash"] == "517458b7423a4f36920d4d95c53ad540"].copy()
data = data[
    [
        "timestamp",
        "AccelerometerUncalibrated_x",
        "AccelerometerUncalibrated_y",
        "AccelerometerUncalibrated_z",
        "GyroscopeUncalibrated_x",
        "GyroscopeUncalibrated_y",
        "GyroscopeUncalibrated_z",
        "MagnetometerUncalibrated_x",
        "MagnetometerUncalibrated_y",
        "MagnetometerUncalibrated_z",
    ]
].copy()
data

,timestamp,AccelerometerUncalibrated_x,AccelerometerUncalibrated_y,AccelerometerUncalibrated_z,GyroscopeUncalibrated_x,GyroscopeUncalibrated_y,GyroscopeUncalibrated_z,MagnetometerUncalibrated_x,MagnetometerUncalibrated_y,MagnetometerUncalibrated_z
1397295,2023-02-28 19:46:28.050,0.611786,-0.021088,0.782272,-0.003351,-0.004160,-0.006677,437.733215,-93.875153,-224.293564
1397296,2023-02-28 19:46:28.060,0.612534,-0.019897,0.781494,-0.002158,-0.002301,-0.005255,437.719116,-93.867264,-224.489243
1397297,2023-02-28 19:46:28.070,0.612762,-0.018875,0.781418,-0.003848,0.000963,-0.005580,437.572144,-93.891785,-224.207077
1397298,2023-02-28 19:46:28.080,0.613327,-0.021088,0.782089,-0.003380,0.000376,-0.006989,437.567780,-93.873169,-224.292389
1397299,2023-02-28 19:46:28.090,0.614822,-0.020981,0.782608,-0.002911,-0.001231,-0.004783,437.674683,-93.943527,-224.201126
...,...,...,...,...,...,...,...,...,...,...
1492291,2023-02-28 20:02:18.010,0.796051,0.101242,0.587524,-0.001657,0.002834,-0.007999,441.561737,-92.924835,-232.037613
1492292,2023-02-28 20:02:18.020,0.798584,0.100723,0.587311,0.000903,0.000042,-0.004031,441.576447,-93.095016,-232.017563
1492293,2023-02-28 20:02:18.030,0.798355,0.097168,0.585419,0.001295,-0.003430,0.000434,441.567047,-93.042694,-232.205032
1492294,2023-02-28 20:02:18.040,0.796768,0.094284,0.582870,0.000393,-0.005154,0.003671,441.648315,-93.167633,-232.078369


In [4]:
# send data via api request
import requests
import gzip
import json

# prepare data for HGBC model
method = "POST"
url = "https://sbar.fuet.ch/HGBC"
payload = data.to_csv(float_format="%.3f", index=False).encode("utf-8")
payload = gzip.compress(payload)
print(f"Payload Size: {len(payload) / 1000000} MB")

# send request
response = requests.request(method, url, data=payload)
text = response.content.decode("utf-8")
text = json.dumps(json.loads(text), indent=2)

# print response
print(f"Response Code: {response.status_code}")
print(text)


Payload Size: 1.685071 MB
Response Code: 200
{
  "0": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufen": 0.0
  },
  "1": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufen": 0.0
  },
  "2": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufen": 0.0
  },
  "3": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufen": 0.0
  },
  "4": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufen": 0.0
  },
  "5": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufen": 0.0
  },
  "6": {
    "Sitzen": 1.0,
    "Laufen": 0.0,
    "Velofahren": 0.0,
    "Rennen": 0.0,
    "Stehen": 0.0,
    "Treppenlaufe

In [5]:
# prepare data for CNN model
method = "POST"
url = "https://sbar.fuet.ch/CNN"
payload = data.to_csv(float_format="%.3f", index=False).encode("utf-8")
payload = gzip.compress(payload)
print(f"Payload Size: {len(payload) / 1000000} MB")

# send request
response = requests.request(method, url, data=payload)
text = response.content.decode("utf-8")
text = json.dumps(json.loads(text), indent=2)

# print response
print(f"Response Code: {response.status_code}")
print(text)

Payload Size: 1.685071 MB
Response Code: 200
{
  "0": {
    "Sitzen": "0.98",
    "Laufen": "0.001",
    "Velofahren": "0.007",
    "Rennen": "0.0",
    "Stehen": "0.012",
    "Treppenlaufen": "0.0"
  },
  "1": {
    "Sitzen": "0.978",
    "Laufen": "0.001",
    "Velofahren": "0.009",
    "Rennen": "0.0",
    "Stehen": "0.013",
    "Treppenlaufen": "0.0"
  },
  "2": {
    "Sitzen": "0.979",
    "Laufen": "0.001",
    "Velofahren": "0.008",
    "Rennen": "0.0",
    "Stehen": "0.012",
    "Treppenlaufen": "0.0"
  },
  "3": {
    "Sitzen": "0.98",
    "Laufen": "0.001",
    "Velofahren": "0.007",
    "Rennen": "0.0",
    "Stehen": "0.012",
    "Treppenlaufen": "0.0"
  },
  "4": {
    "Sitzen": "0.978",
    "Laufen": "0.001",
    "Velofahren": "0.009",
    "Rennen": "0.0",
    "Stehen": "0.013",
    "Treppenlaufen": "0.0"
  },
  "5": {
    "Sitzen": "0.98",
    "Laufen": "0.001",
    "Velofahren": "0.007",
    "Rennen": "0.0",
    "Stehen": "0.012",
    "Treppenlaufen": "0.0"
  },
  "6": {